In [4]:
import pandas as pd
import re
import string
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import json
from nltk.tokenize import TweetTokenizer
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.util import ngrams
#from google.colab import drive
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy import sparse
import os
import warnings
from stance_utils import *
warnings.filterwarnings('ignore')
import time
import time
start_time = time.time()

# train_data_file = '/data/parush/stance_mohammed/new_train.txt'
# test_data_file = '/data/parush/stance_mohammed/new_test.txt'
# TARGETS = [ 'Atheism','Climate Change is a Real Concern', 'Feminist Movement','Hillary Clinton', 'Legalization of Abortion', 'Donald Trump']


train_data_file = '/data/parush/SomasundaranWiebe-politicalDebates/train.txt'
test_data_file = '/data/parush/SomasundaranWiebe-politicalDebates/test.txt'
TARGETS = ['god','healthcare','guns','gayRights','abortion', 'creation']


# train_data_file = '/data/parush/Data_MPCHI/train.txt'
# test_data_file = '/data/parush/Data_MPCHI/test.txt'
# TARGETS = ['Are E-Cigarettes safe?','Does MMR Vaccine lead to autism in children?',
#       'Does Sunlight exposure lead to skin cancer?','Does Vitamin C prevent common cold?',
#       'Should women take HRT post-menopause?']

target = TARGETS[0]

df = pd.read_csv(train_data_file, sep='\t')
df1 = pd.read_csv(test_data_file, sep='\t')
vectorizer = 'tfidf'   # set 'count' or 'tfidf'
analyzer = 'both'  # set 'word' or 'both' ( word and char)
if vectorizer == 'count':
    if analyzer == 'word':
        vectorizer = CountVectorizer(analyzer='word',ngram_range=(1,1))
    else:
        vectorizer = CountVectorizer(analyzer='word',ngram_range=(1,3))
        char_vectorizer = CountVectorizer(analyzer='char',ngram_range=(2,5))
else:
    if analyzer == 'word':
        vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,1))
    else:
        vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,3))
        char_vectorizer = TfidfVectorizer(analyzer='char',ngram_range=(2,5))
        
#List of FAVOR Tweets
def get_training_data_and_labels(file,target):
    print("Working on target:", target)
    
    train_corpus = []
    train_labels = []
    classes = {'FAVOR': 0, 'AGAINST': 1, 'NONE': 2}
    with open(file) as f:
        for line in f:
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
            #if line[0].strip() != 'ID' and target in line[1].strip():
            if line[0].strip() != 'ID':  #Uncomment this line if training on wholedataset and comment the line above.
                
                tweet = line[2]
                stance = line[3]
                
                train_corpus.append(tweet)
                train_labels.append(classes[stance])

    
    if analyzer == 'word':
        ngram_vectorized_data = vectorizer.fit_transform(train_corpus)
        
        return ngram_vectorized_data, train_labels
    else:
        ngram_vectorized_data = vectorizer.fit_transform(train_corpus)
        char_vectorized_data = char_vectorizer.fit_transform(train_corpus)
        l = np.hstack((ngram_vectorized_data.toarray(), char_vectorized_data.toarray()))
        train_vectorized_data = sparse.csr_matrix(l)
        
        return train_vectorized_data, train_labels 
       
#preparing test_data
def get_test_data_and_labels(file,target):
    print("Working on target:", target)
    test_corpus = []
    test_labels = []
    classes = {'FAVOR': 0, 'AGAINST': 1, 'NONE': 2}
    with open(file) as f:
        for line in f:
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
        #if line[0].strip() != 'ID' and line[3].strip() == 'FAVOR' and line[1] == t:
            #if line[0].strip() != 'ID' and target in line[1].strip():
            if line[0].strip() != 'ID': #Uncomment this line if testing on wholedataset and comment the line above.
                tweet = line[2]
                stance = line[3]
                #print(line[0])
                test_corpus.append(tweet)
                test_labels.append(classes[stance])
    if analyzer == 'word':
        test_ngram_vectorized_data = vectorizer.transform(test_corpus)
        
        return test_ngram_vectorized_data, test_labels
    else:
        test_ngram_vectorized_data = vectorizer.transform(test_corpus)
        test_char_vectorized_data = char_vectorizer.transform(test_corpus)
        l2 = np.hstack((test_ngram_vectorized_data.toarray(), test_char_vectorized_data.toarray()))
        test_vectorized_data = sparse.csr_matrix(l2)
        
        
        return test_vectorized_data,test_labels
    
    
    
X_train, y_train =  get_training_data_and_labels(train_data_file, target)
X_test, y_test = get_test_data_and_labels(test_data_file,target)  
from sklearn.model_selection import StratifiedKFold
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
scores = ['precision', 'recall']


for score in scores:
    
    print("# Tuning hyper-parameters for %s" % score)
    print()
    cv = StratifiedKFold(n_splits=2, shuffle=True, random_state = 2 )
    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score, cv = cv
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred, digits = 4,labels = [0,1]))
    print()
print("--- %s seconds ---" % (time.time() - start_time))

Working on target: god
Working on target: god
# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

Grid scores on development set:

0.276 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.276 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.276 (+/-0.000) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.276 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.636 (+/-0.035) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.276 (+/-0.000) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.613 (+/-0.000) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.636 (+/-0.034) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.616 (+/-0.002) for {'C': 1, 'kernel': 'linear'}
0.613 (+/-0.003) for {'C': 10, 'kernel': 'linear'}
0.592 (+/-0.019) for {'C': 100, 'kernel': 'linear'}
0.586 (+/-0.007) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:


              precision  

## Test on another target.
